# Noob2Vec

## Imports

In [7]:
from __future__ import absolute_import, division, print_function

In [8]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re
import csv
import string

In [9]:
import nltk
from nltk.util import ngrams
import gensim.models.word2vec as w2v
from gensim.models import Phrases
from collections import Counter

import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [13]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/shu/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['seed']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


**Set up logging**

In [14]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

**Download NLTK tokenizer models (only the first time)**

In [15]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /Users/shu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/shu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Prepare Corpus

**Load books from files**

In [16]:
data_filenames = sorted(glob.glob("data/*.txt"))

In [17]:
print("Found books:",data_filenames)


Found books: ['data/*.txt']


**Combine the books into one string**

In [18]:
corpus_raw = u""
for data_filename in data_filenames:
    print("Reading '{0}'...".format(data_filename))
    with codecs.open(data_filename, "r", "utf-8") as data_file:
        corpus_raw += data_file.read()
    print("Corpus is now {0} characters long".format(len(corpus_raw)))
    print()

Reading 'data/*.txt'...
Corpus is now 194670485 characters long



**Split the corpus into sentences**

In [19]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [20]:
raw_sentences = tokenizer.tokenize(corpus_raw)

In [21]:
partial_raw_sentences = raw_sentences[0:50] 

In [1]:
#partial_raw_sentences

In [22]:
from nltk.corpus import stopwords 
stopset = set(stopwords.words('english'))

In [ ]:
# cleanups = [token.lower() for token in raw_sentences if token.lower() not in stopset and len(token)>2]

In [23]:
# split the sentense to words level and cleanup stopwords from raw_sentences, 
cleanups = []
for token in partial_raw_sentences:
    cleanup = []
    lower_token = token.lower()
    words_list = lower_token.split()
    for word in words_list:
        if word not in stopset:
            cleanup.append(word)
    cleanups.append(cleanup)

In [24]:
#testing for cleaning the stopword
partial_raw_sentences[0]

'body\n"nah fam we good\n\n"\n"ok but without caffeine my friend isn\'t motivated to even go out for a run, he just gets sleepy.'

In [25]:
cleanups[0]

['body',
 '"nah',
 'fam',
 'good',
 '"',
 '"ok',
 'without',
 'caffeine',
 'friend',
 'motivated',
 'even',
 'go',
 'run,',
 'gets',
 'sleepy.']

In [26]:
len(cleanups)

50

In [27]:
#count for the token
token_count = sum([len(cleanup1) for cleanup1 in cleanups])
print("The book corpus contains {0:,} tokens".format(token_count))

The book corpus contains 497 tokens


In [28]:
#??? still a bit confused about the phrases function and the bigram data type
from gensim.models.phrases import Phraser
phrases = Phrases(cleanups)
bigram = Phraser(phrases)

2018-01-18 14:20:54,049 : INFO : collecting all words and their counts
2018-01-18 14:20:54,052 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2018-01-18 14:20:54,056 : INFO : collected 797 word types from a corpus of 497 words (unigram + bigrams) and 50 sentences
2018-01-18 14:20:54,059 : INFO : using 797 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
2018-01-18 14:20:54,063 : INFO : source_vocab length 797
2018-01-18 14:20:54,073 : INFO : Phraser built with 0 0 phrasegrams


In [29]:
trigram = Phrases(bigram[cleanups])

2018-01-18 14:20:56,804 : INFO : collecting all words and their counts
2018-01-18 14:20:56,816 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2018-01-18 14:20:56,827 : INFO : collected 797 word types from a corpus of 497 words (unigram + bigrams) and 50 sentences
2018-01-18 14:20:56,828 : INFO : using 797 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>


In [30]:
sent = [u'fleur', u'de', u'peau', u'new', u'york', u'lunar', u'elite', u'hello', u'me']
>>> print((trigram[bigram[sent]]))

['fleur', 'de', 'peau', 'new', 'york', 'lunar', 'elite', 'hello', 'me']


/Users/shu/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [31]:
new_stuff= list((trigram[bigram[cleanups]]))

/Users/shu/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [63]:
new_stuff

[['body',
  '"nah',
  'fam',
  'good',
  '"',
  '"ok',
  'without',
  'caffeine',
  'friend',
  'motivated',
  'even',
  'go',
  'run,',
  'gets',
  'sleepy.'],
 ['maybe', 'overdid', 'speed?'],
 ['mean',
  'noticed',
  'performance',
  'improvement',
  'caffeine',
  'compared',
  'stuff,',
  'feels',
  'useless',
  'without',
  'stimulants.'],
 ["can't",
  'get',
  'stuff',
  'done,',
  'brain',
  'work',
  "what's",
  'good',
  'balance?'],
 ['drop', 'amphetamines?'],
 ['caffeine', 'pill', 'better', 'cup', 'coffee?"'],
 ['"when',
  'xc',
  'coach',
  'said',
  'eat',
  'running',
  'cause',
  'would',
  'get',
  'cramps',
  'eat',
  '4',
  'hours',
  'running',
  'amphetamine',
  'powder,',
  'kind',
  'yellowish-brown,',
  'think',
  'mostly',
  'pure.'],
 ['eating', 'running', 'work', 'you?'],
 ['usually',
  'eat',
  'long',
  'runs"',
  'barefoot',
  'sounds....',
  'interesting',
  'o_o',
  'friend',
  'similar',
  'story',
  'except',
  'winter',
  'alaska',
  'retarded',
  'boy'

## Train Word2Vec

## Train Word2Vec

In [61]:
# Dimensionality of the resulting word vectors.
num_features = 300

# Minimum word count threshold.
min_word_count = 5

# Number of threads to run in parallel.
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 15

# Downsample setting for frequent words.
downsampling = 1e-3

# Seed for the RNG, to make the results reproducible.
seed = 1

In [72]:
model2vec = w2v.Word2Vec(
    sg=0,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

In [63]:
model2vec.build_vocab(new_stuff)

2018-01-18 15:47:18,283 : INFO : collecting all words and their counts
2018-01-18 15:47:18,287 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-01-18 15:47:18,289 : INFO : collected 363 word types from a corpus of 497 raw words and 50 sentences
2018-01-18 15:47:18,291 : INFO : Loading a fresh vocabulary
2018-01-18 15:47:18,294 : INFO : min_count=5 retains 7 unique words (1% of original 363, drops 356)
2018-01-18 15:47:18,296 : INFO : min_count=5 leaves 46 word corpus (9% of original 497, drops 451)
2018-01-18 15:47:18,299 : INFO : deleting the raw counts dictionary of 363 items
2018-01-18 15:47:18,300 : INFO : sample=0.001 downsamples 7 most-common words
2018-01-18 15:47:18,302 : INFO : downsampling leaves estimated 4 word corpus (9.0% of prior 46)
2018-01-18 15:47:18,305 : INFO : estimated required memory for 7 words and 300 dimensions: 20300 bytes
2018-01-18 15:47:18,308 : INFO : resetting layer weights


In [64]:
print("Word2Vec vocabulary length:", len(model2vec.wv.vocab))

Word2Vec vocabulary length: 7


**Start training, this might take a minute or two...**

In [73]:
model2vec.train(new_stuff, total_examples=model2vec.corpus_count, epochs=model2vec.iter)

AttributeError: 'Word2Vec' object has no attribute 'corpus_count'

**Save to file, can be useful later**

In [67]:
if not os.path.exists("trained"):
    os.makedirs("trained")

In [68]:
model2vec.save(os.path.join("trained", "guitar1.w2v"))

2018-01-18 15:48:25,352 : INFO : saving Word2Vec object under trained/guitar1.w2v, separately None
2018-01-18 15:48:25,355 : INFO : not storing attribute syn0norm
2018-01-18 15:48:25,358 : INFO : not storing attribute cum_table
2018-01-18 15:48:25,364 : INFO : saved trained/guitar1.w2v


In [69]:
model2vec.wv.save_word2vec_format('guitar.bin', binary=True)

2018-01-18 15:48:26,268 : INFO : storing 7x300 projection weights into guitar.bin


## Explore the trained model.

In [71]:
model2vec = w2v.Word2Vec.load(os.path.join("trained", "furniture.w2v"))

2018-01-18 15:49:37,128 : INFO : loading Word2Vec object from trained/furniture.w2v


FileNotFoundError: [Errno 2] No such file or directory: 'trained/furniture.w2v'

In [46]:
from sklearn.decomposition import PCA, FastICA
import hdbscan

In [47]:
class Decomposer():
    def __init__(self, n_components=15, copy=True, whiten=True, mode="pca", max_iter=2000, model=None):
        if model is not None:
            self.model = model
        else:
            if mode == "pca":
                self.model = PCA(n_components=n_components, copy=copy, whiten=whiten)
            elif mode == "ica":
                self.model = FastICA(n_components=n_components, whiten=whiten, max_iter=max_iter)
    def fit(self, X):
        self.model.fit(X)

    def transform(self, X):
        return self.model.transform(X)

    def fit_transform(self, X):
        data = self.model.fit_transform(X)
        return data

In [48]:
decomposer = Decomposer(n_components=15)
X_decomp = decomposer.fit_transform(model2vec.wv.syn0)

In [49]:
class HDBscanClusterer():
    _min_cluster_size = 15
    _metric = "euclidean"

    def __init__(self, min_cluster_size=15, min_samples=1, alpha=1.0, metric="euclidean"):
        self.min_cluster_size = min_cluster_size
        self.min_samples = min_samples
        self.alpha = alpha
        self.metric = metric
        self.labels = []

    @property
    def min_cluster_size(self):
        return self._min_cluster_size

    @min_cluster_size.setter
    def min_cluster_size(self, value):
        self._min_cluster_size = value

    @property
    def metric(self):
        return self._metric

    @metric.setter
    def metric(self, value):
        self._metric = value

    @property
    def labels(self):
        return self._labels

    @labels.setter
    def labels(self, value):
        self._labels = value

    def cluster(self, X):
        model = hdbscan.HDBSCAN(min_cluster_size=self.min_cluster_size, min_samples=self.min_samples, alpha=self.alpha, metric=self.metric)
        labels = model.fit_predict(X)
        self.labels = [{'index': i, 'cluster': j} for i, j in enumerate(labels)]


In [50]:
from sklearn.preprocessing import normalize
class L2Normalizer():
    def norm(self, X):
        return normalize(X, norm='l2')


In [51]:
X_decomp_norm = L2Normalizer().norm(X_decomp)

In [52]:
df = pd.DataFrame(X_decomp_norm)
df.head(10)

,0,1,2,3,4,5,6
0,-0.648851,-0.342814,-0.352010,0.257205,-0.320311,0.161088,0.377965
1,-0.137573,0.588912,-0.076517,0.034465,0.466079,0.516843,0.377964
2,-0.174164,-0.427684,0.376064,-0.066838,0.620961,-0.335280,0.377964
3,0.675519,-0.449068,-0.146561,0.050394,-0.030337,0.417391,0.377964
4,0.098404,0.174321,-0.521193,-0.597560,-0.062925,-0.429409,0.377964
5,0.245488,0.319209,0.063182,0.669410,-0.159364,-0.466353,0.377965
6,-0.058824,0.137124,0.657035,-0.347077,-0.514103,0.135719,0.377964


In [53]:
hdb = HDBscanClusterer(min_cluster_size=8, min_samples=1, alpha=0.25)
hdb.cluster(X_decomp_norm)

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=1, alpha=0.25).fit(X_decomp_norm)
clusterer.condensed_tree_.plot(select_clusters=True,
selection_palette=sns.color_palette('deep', 8))

In [55]:
df = pd.DataFrame(X_decomp_norm)

In [56]:
len(df)

7

In [ ]:
res = hdb.labels
vocab_map = model2vec.wv.vocab
df = pd.DataFrame(res)
df = df[df['cluster'] > -1]
counts = df['cluster'].value_counts().to_dict()
good = {k: v for k, v in counts.items() if v < 1000}
df['ok'] = df['cluster'].apply(lambda x: x in good)
df = df[df['ok']]
n_clusters = len(df['cluster'].unique())
print("FOUND {} CLUSTERS".format(n_clusters))

In [ ]:
data_df = pd.DataFrame.from_dict(model2vec.wv.vocab, orient='index', dtype=None)
data_df.head(10)


In [ ]:
df['raw_vector'] = df['index'].apply(lambda x: model2vec.wv.syn0[x])
# df['word'] = df['index'].apply(lambda x: model2vec.wv.vocab[x])
df['decomp_norm_vector'] = df['index'].apply(lambda x: X_decomp_norm[x])
len(df.index)
df.head(300)
len(df)

In [ ]:
df.to_csv('furniturehdb1.csv')

In [ ]:
df.head(300)

In [ ]:
for w in model2vec.wv.vocab:
     print(w,model2vec.wv.vocab[w].count)

In [ ]:
data_df = pd.DataFrame.from_dict(model2vec.wv.vocab, orient='index', dtype=None)

In [ ]:
data_df.to_csv('guitarwords.csv')

## Train Word2Vec

### Compress the word vectors into 2D space and plot them

In [ ]:
tsne = sklearn.manifold.TSNE(n_components=2, random_state=0, perplexity=5.0, early_exaggeration=12.0, learning_rate=200.0, n_iter=1000, n_iter_without_progress=300)

In [ ]:
tsne = sklearn.manifold.TSNE(n_components=2, random_state=0)

In [ ]:
all_word_vectors_matrix = model2vec.wv.syn0

**Train t-SNE, this could take a minute or two...**

In [ ]:
all_word_vectors_matrix_2d = tsne.fit_transform(all_word_vectors_matrix)

In [ ]:
from sklearn.cluster import KMeans;

**Plot the big picture**

In [ ]:
points = pd.DataFrame(
    [
        (word, coords[0], coords[1])
        for word, coords in [
            (word, all_word_vectors_matrix_2d[model2vec.wv.vocab[word].index])
            for word in model2vec.wv.vocab
        ]
    ],
    columns=["word", "x", "y"]
)

In [ ]:
pointsforcolor = pd.DataFrame(
    [
        (coords[0], coords[1])
        for coords in [
            (all_word_vectors_matrix_2d[model2vec.wv.vocab[word].index])
            for word in model2vec.wv.vocab
        ]
    ],
    columns=["x", "y"]
)

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=100)
kmeans.fit(pointsforcolor)
y_kmeans = kmeans.predict(pointsforcolor)
pointsforcolor.plot.scatter("x", "y", c=y_kmeans, s=3, cmap='viridis', figsize=(20, 15))
points['kmeans'] = y_kmeans

In [ ]:
pointsforcolor.head(10)
points.to_csv('ps4coords.csv')

In [ ]:
sns.set_context("poster")

In [ ]:
points.plot.scatter("x", "y", s=10, figsize=(20, 15))
plt.savefig('books_read.png')

**Zoom in to some interesting places**

In [ ]:
points.to_csv('paintcoords.csv')

In [ ]:
def plot_region(x_bounds, y_bounds):
    slice = points[
        
        (x_bounds[0] <= points.x) &
        (points.x <= x_bounds[1]) & 
        (y_bounds[0] <= points.y) &
        (points.y <= y_bounds[1])
    ]
    
    ax = slice.plot.scatter("x", "y", s=15, figsize=(10, 8))
    for i, point in slice.iterrows():
        ax.text(point.x + 0.005, point.y + 0.005, point.word, fontsize=7)

In [ ]:
plot_region(x_bounds=(-30, -10), y_bounds=(-30, -10))

In [ ]:
plot_region(x_bounds=(0, 10), y_bounds=(40, 45))

### Explore semantic similarities between book characters

**Words closest to the given word**

In [ ]:
model2vec.most_similar("bad",[],100)

In [ ]:
model2vec.most_similar("xbox",[],100)

In [ ]:
model2vec.most_similar("xbox")

**Linear relationships between word pairs**

In [ ]:
def nearest_similarity_cosmul(start1, end1, end2):
    similarities = model2vec.most_similar_cosmul(
        positive=[end2, start1],
        negative=[end1]
    )
    start2 = similarities[0][0]
    print("{start1} is related to {end1}, as {start2} is related to {end2}".format(**locals()))
    return start2

In [ ]:
nearest_similarity_cosmul("lather", "foamy", "fragance")